In [1]:
import glob
from multiprocessing import Process
import os
import re

import numpy as np
import PIL
from PIL import Image
from random import normalvariate
from random import randint

import time

from keras.applications.inception_v3 import InceptionV3

from keras_preprocessing import image as Kimage
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten,Conv2D

from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
#initialise the list

training_images = []
training_labels = []
testing_images = []
testing_labels = []
validation_images = []
validation_labels= []
current_label = 0

## image preprocessing 图像预处理

In [7]:

#rescale the image into a side with in [256,480]
def rescale(image):
    w = image.size[0]
    h = image.size[1]
    sizeMax =350
    sizeMin =300
    random_size = randint(sizeMin,sizeMax)
    if w < h:
        return image.resize((random_size,round(h/w * random_size)))
    else:
        return image.resize((round(w/h * random_size),random_size))

# crop the image randomly
def random_crop(image):
    w = image.size[0]
    h = image.size[1]
    size =299
    new_left = randint(0,w - size)
    new_upper = randint(0,h - size)
    return image.crop((new_left,new_upper,size+new_left,size+new_upper))

# flip the image horizontally
def horizontal_flip(image):
    return image.transpose(Image.FLIP_LEFT_RIGHT)

# image nomalized
def nomalizing(image,mean_value,add_num):
    image = np.array(image)
    for i in range(3):
        add_num = add_num.astype(int)
        image[:,:,i] = (image[:,:,i] - mean_value[i] + add_num[i])#/std[i]
    return image

# get the mean of the pixel value of the image
def mean(image_dir):
    image_lst = Kimage.list_pictures(image_dir)
    for pic in image_lst:
        image = Kimage.load_img(pic)
        image = Kimage.img_to_array(image)
        try:
            image_array = np.concatenate((image_array,image),0)# 第一张图片不存在image_array
        except:
            image_array = image
    mean_value = image_array.mean(0)
    # std_value = image_array.std(0)
    return mean_value


# find pca
def pca(image_dir, mean_value):
    image_lst = Kimage.list_pictures(image_dir)
    for pic in image_lst:
        image = Kimage.load_img(pic)
        image = Kimage.img_to_array(image)
        image -= mean_value
        try:
            image_array = np.concatenate((image_array,image),0)# 第一张图片不存在image_array
        except:
            image_array = image
        image_cov = np.cov([image_array[:,0],image_array[:,1],image_array[:,2]])
        lambd,p = np.linalg.eig(image_cov)
        alpha0 = normalvariate(0,0.1)
        alpha1 = normalvariate(0,0.1)    
        alpha2 = normalvariate(0,0.1)
        v = np.transpose((alpha0*lambd[0],alpha1*lambd[1],alpha2*lambd[2]))
        add_num = np.dot(v,np.transpose(p))
    return add_num

## 载入图像 load the image

In [13]:
validation_percentage = 15
testing_percentage = 15
#define the input folder name
Input_data_dir = 'images'
image_lst = Kimage.list_pictures("images")

for pic in image_lst:
    ldimage = Kimage.load_img(pic)
    if 'AJ' in pic:
        current_label == 0
    elif 'BX' in pic:
        current_label == 1
    elif 'CJ' in pic:
        current_label == 2
    elif 'CK' in pic:
        current_label == 3
    elif 'CQ' in pic:
        current_label == 4
    elif 'CR' in pic:
        current_label == 5
    elif 'MWT' in pic:
        current_label == 6
    elif 'PL' in pic:
        current_label == 7
    elif 'QF' in pic:
        current_label == 8
    elif 'SG' in pic:
        current_label == 9
    elif 'SL' in pic:
        current_label == 10
    elif 'TJ' in pic:
        current_label == 11
    elif 'TL' in pic:
        current_label == 12
    else:
        current_label = 13
    ldimage = Kimage.img_to_array(ldimage)
    ldimage = rescale(ldimage)
    

       
    chance = np.random.randint(100)
    #按概率随机分到三个数据集中
    if chance < testing_percentage:
        testing_images.append(image_value)
        testing_labels.append(current_label)
    elif chance < (testing_percentage + validation_percentage):
        validation_images.append(image_value)
        validation_labels.append(current_label)
    else:
        training_images.append(image_value)
        training_labels.append(current_label)
            
    if chance >= testing_percentage:
        
        for i in range(5):
            training_images.append(random_crop(ldimage))
            training_labels.append(current_label)
        
        ldimage_1 = random_crop(ldimage)
        ldimage_2 = random_crop(ldimage)
        ldimage_3 = random_crop(ldimage)
        ldimage_4 = random_crop(ldimage)
        ldimage_5 = random_crop(ldimage)
        ldimage_6 = ldimage.resize(299,299)

        mean_value = mean("images")
        PCA_v = pca("images",mean_value)
        ldimage_7 = nomalizing(ldimage_1)
        ldimage_8 = nomalizing(ldimage_2)
        ldimage_9 = nomalizing(ldimage_3)
        ldimage_10 = nomalizing(ldimage_4)
        ldimage_11 = nomalizing(ldimage_5)
        ldimage_12 = nomalizing(ldimage_6)
        
    for i in (1,13):
        
        training_images.append()
    


#将训练数据和标签以同样的方式打乱
state = np.random.get_state()
np.random.shuffle(training_images)
np.random.set_state(state)
np.random.shuffle(training_labels)

    